# Assignment 3

In [1]:
# Loading the libraries
import pandas as pd
import numpy as np
from surprise import KNNWithMeans,Dataset, accuracy,Reader
from surprise.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
# reading all the files
ratings = pd.read_csv("ratings.csv")
movies = pd.read_csv("movies.csv")
links = pd.read_csv("links.csv")
tags = pd.read_csv("tags.csv")

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings.shape

(100836, 4)

In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.shape

(9742, 3)

In [7]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
links.shape

(9742, 3)

In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [10]:
tags.shape

(3683, 4)

## 1. Create recommender system using ratings.csv file

In [11]:
#reader class to mention the rating scale
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(ratings[["userId","movieId","rating"]],reader)

In [12]:
#divide dataset in trainset and test set
[trainset,testset] = train_test_split(data,test_size=.15,shuffle=True)
recom = KNNWithMeans(k=40, sim_options={"Name":"Cosine","user_based":True})
recom.fit(data.build_full_trainset())
test_pred = recom.test(testset)
RMSE = accuracy.rmse(test_pred)
print("RMSE =",RMSE)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7026
RMSE = 0.7025607203775674


## 2. Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input)

In [13]:
userID = int(input("USER ID = "))
movieID = int(input("MOVIE ID = "))
print("The rating value for USER ID:",userID,"and","MOVIE ID:",movieID,"=",round((recom.predict(userID,movieID)[3]),2))

USER ID = 508
MOVIE ID = 639
The rating value for USER ID: 508 and MOVIE ID: 639 = 1.24


## 3. Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions).Take userID as a keyboard input. Fix the threshold rating as 2.5

In [33]:
# creating empty dictionary -> userID as key and movieID as value
movie_list = {}
userID = int(input("USER ID = "))

for movieID in movies["movieId"]:
    if recom.predict(userID,movieID)[3] > 2.5:
        movie_list[movieID] = recom.predict(userID,movieID)[3]

# finding 10 highest values in a Dictionary    
from collections import Counter
k = Counter(movie_list)
high = k.most_common(10)

print("Top 10 movies for user",userID,"are listed below")
for i in range(10):
    print(i+1,"\t",list(movies[movies["movieId"]==high[i][0]]["title"])[0])

USER ID = 508
Top 10 movies for user 508 are listed below
1 	 The Big Bus (1976)
2 	 Seve (2014)
3 	 Unfaithfully Yours (1948)
4 	 Holy Mountain, The (Montaña sagrada, La) (1973)
5 	 On the Beach (1959)
6 	 Beloved (1998)
7 	 Strictly Sexual (2008)
8 	 Babes in Toyland (1934)
9 	 Dad's Army (1971)
10 	 High and Low (Tengoku to jigoku) (1963)


## 4. Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame

In [34]:
df = pd.merge(movies[["movieId","genres"]],tags[["movieId","tag"]],how="left",on="movieId")
df1 = pd.merge(df,links[["movieId","imdbId"]],how="left",on="movieId")

In [35]:
df1.head()

,movieId,genres,tag,imdbId
0,1,Adventure|Animation|Children|Comedy|Fantasy,pixar,114709
1,1,Adventure|Animation|Children|Comedy|Fantasy,pixar,114709
2,1,Adventure|Animation|Children|Comedy|Fantasy,fun,114709
3,2,Adventure|Children|Fantasy,fantasy,113497
4,2,Adventure|Children|Fantasy,magic board game,113497


In [36]:
ls = []
for i in range(10):
    ls.append(high[i][0])
df_final = df1[df1["movieId"].isin(ls)]
df_final

,movieId,genres,tag,imdbId
2438,2314,Drama,NaN,120603
3181,3086,Children|Comedy|Fantasy|Musical,NaN,24852
3412,3379,Drama,NaN,53137
5006,5490,Action|Comedy,NaN,74205
6350,7926,Crime|Drama|Film-Noir|Thriller,In Netflix queue,57565
6770,25947,Comedy,NaN,40919
6824,26326,Drama,NaN,71615
8589,67618,Comedy|Drama|Romance,NaN,494277
10831,132333,Documentary|Drama,NaN,3149640
11273,152105,Comedy,NaN,66963


In [37]:
# finding average user rating
avg_usr_rating = {}
for i in df_final["movieId"]:
    avg_usr_rating[i] = round((ratings[ratings["movieId"]==i]['rating'].mean()),2)
avg_usr_rating

{2314: 4.5,
 3086: 5.0,
 3379: 4.5,
 5490: 5.0,
 7926: 4.0,
 25947: 5.0,
 26326: 4.5,
 67618: 5.0,
 132333: 5.0,
 152105: 4.5}

In [38]:
id_rating = [(x,y) for x,y in avg_usr_rating.items() if x in ls]
avg_rating = []
for i in range(10):
    avg_rating.append(id_rating[i][1])

df_final["avg_usr_rating"] = avg_rating

# printing MovieID, IMDB ID,Average User Rating,genres and tag for all the movies in step 3
df_final

,movieId,genres,tag,imdbId,avg_usr_rating
2438,2314,Drama,NaN,120603,4.5
3181,3086,Children|Comedy|Fantasy|Musical,NaN,24852,5.0
3412,3379,Drama,NaN,53137,4.5
5006,5490,Action|Comedy,NaN,74205,5.0
6350,7926,Crime|Drama|Film-Noir|Thriller,In Netflix queue,57565,4.0
6770,25947,Comedy,NaN,40919,5.0
6824,26326,Drama,NaN,71615,4.5
8589,67618,Comedy|Drama|Romance,NaN,494277,5.0
10831,132333,Documentary|Drama,NaN,3149640,5.0
11273,152105,Comedy,NaN,66963,4.5
